In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
class UniversalPerceptron(tf.keras.Model):
    def __init__(self, input_size, output_size, task_type='binary_classification'):
        super(UniversalPerceptron, self).__init__()
        self.fc = tf.keras.layers.Dense(output_size)

        if task_type == 'binary_classification':
            self.activation = tf.keras.activations.sigmoid
            self.loss_function = tf.keras.losses.BinaryCrossentropy()
        elif task_type == 'multi_class_classification':
            self.activation = tf.keras.activations.softmax
            self.loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
        elif task_type == 'regression':
            self.activation = tf.keras.activations.linear
            self.loss_function = tf.keras.losses.MeanSquaredError()
        else:
            raise ValueError("Invalid task_type. Supported types are 'binary_classification', 'multi_class_classification', and 'regression'.")

    def call(self, inputs):
        x = self.fc(inputs)
        return self.activation(x)

In [ ]:
# Example usage
if __name__ == "__main__":
    # Generate some random training data
    np.random.seed(42)
    input_size = 2
    data_size = 100
    inputs = np.random.rand(data_size, input_size).astype(np.float32)
    labels_binary_cls = np.random.randint(2, size=(data_size, 1)).astype(np.float32)
    labels_multi_cls = np.random.randint(3, size=(data_size)).astype(np.int32)
    labels_regression = np.random.rand(data_size, 1).astype(np.float32)

    # Create a UniversalPerceptron model for binary classification
    binary_cls_model = UniversalPerceptron(input_size, 1, task_type='binary_classification')

    # Define optimizer
    binary_cls_optimizer = tf.optimizers.SGD(learning_rate=0.01)

    # Training for binary classification
    for epoch in range(100):
        with tf.GradientTape() as tape:
            outputs = binary_cls_model(inputs)
            loss = binary_cls_model.loss_function(labels_binary_cls, outputs)
        
        gradients = tape.gradient(loss, binary_cls_model.trainable_variables)
        binary_cls_optimizer.apply_gradients(zip(gradients, binary_cls_model.trainable_variables))

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/100], Loss: {loss.numpy():.4f}')

    # Testing the trained model for binary classification
    test_inputs = np.random.rand(5, input_size).astype(np.float32)
    binary_cls_predictions = (binary_cls_model(test_inputs) >= 0.5).numpy()
    print("Binary Classification Predictions:", binary_cls_predictions)

    # Create a UniversalPerceptron model for multi-class classification
    multi_cls_model = UniversalPerceptron(input_size, 3, task_type='multi_class_classification')

    # Define optimizer
    multi_cls_optimizer = tf.optimizers.SGD(learning_rate=0.01)

    # Training for multi-class classification
    for epoch in range(100):
        with tf.GradientTape() as tape:
            outputs = multi_cls_model(inputs)
            loss = multi_cls_model.loss_function(labels_multi_cls, outputs)

        gradients = tape.gradient(loss, multi_cls_model.trainable_variables)
        multi_cls_optimizer.apply_gradients(zip(gradients, multi_cls_model.trainable_variables))

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/100], Loss: {loss.numpy():.4f}')

    # Testing the trained model for multi-class classification
    multi_cls_predictions = np.argmax(multi_cls_model(test_inputs), axis=1)
    print("Multi-Class Classification Predictions:", multi_cls_predictions)

    # Create a UniversalPerceptron model for regression
    regression_model = UniversalPerceptron(input_size, 1, task_type='regression')

    # Define optimizer
    regression_optimizer = tf.optimizers.SGD(learning_rate=0.01)

    # Training for regression
    for epoch in range(100):
        with tf.GradientTape() as tape:
            outputs = regression_model(inputs)
            loss = regression_model.loss_function(labels_regression, outputs)

        gradients = tape.gradient(loss, regression_model.trainable_variables)
        regression_optimizer.apply_gradients(zip(gradients, regression_model.trainable_variables))

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/100], Loss: {loss.numpy():.4f}')

    # Testing the trained model for regression
    regression_predictions = regression_model(test_inputs).numpy()
    print("Regression Predictions:", regression_predictions)